In [1]:
#Importing the libraries
import pandas as pd
import numpy as np

In [2]:
#Mount the drive 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#Load the dataset
df=pd.read_csv("drive/My Drive/CSV_Files/DrugsDotCom_preprocessed.csv",encoding="ISO-8859-1")
df.head()

,Unnamed: 0,index,drugName,condition,review,rating,date,usefulCount,cleaned_review,tokenized,lemmatized
0,0,0,Liraglutide,Obesity,""" began with the minimum 0.6 mg dose of Saxend...",10,"March 18, 2017",28,[ began minimum mg dose saxenda february incre...,"['[', 'began', 'minimum', 'mg', 'dose', 'saxen...",[ began minimum mg dose saxenda february incre...
1,1,1,Quetiapine,Bipolar Disorde,""" Caused depression and negative, self defeati...",1,"August 25, 2016",2,[ caused depression negative self defeating th...,"['[', 'caused', 'depression', 'negative', 'sel...",[ caused depression negative self defeating th...
2,2,2,Ethinyl estradiol / levonorgestrel,Birth Control,""" Ever since I started taking this birth contr...",1,"October 30, 2017",2,[ ever since started taking birth control noti...,"['[', 'ever', 'since', 'started', 'taking', 'b...",[ ever since started taking birth control noti...
3,3,3,Lorazepam,Anxiety,""" Had a sort of nervous breakdown during a tri...",9,"June 4, 2014",64,[ sort nervous breakdown trip could catch brea...,"['[', 'sort', 'nervous', 'breakdown', 'trip', ...",[ sort nervous breakdown trip could catch brea...
4,4,4,Aczone,Acne,""" I am 28, have struggled with acne for 10 yea...",10,"June 2, 2015",16,[ struggled acne year product kept consistentl...,"['[', 'struggled', 'acne', 'year', 'product', ...",[ struggled acne year product kept consistentl...


In [4]:
#Set the col width
pd.set_option('max_rows', 99999)
pd.set_option('max_colwidth', 400)
pd.describe_option('max_colwidth')

display.max_colwidth : int or None
    The maximum width in characters of a column in the repr of
    a pandas data structure. When the column overflows, a "..."
    placeholder is embedded in the output. A 'None' value means unlimited.
    [default: 50] [currently: 400]


In [5]:
#Droping duplicates
df=df.drop_duplicates(subset=["drugName","condition","review","rating"],keep='first')

In [6]:
#reseting the index
df = df.copy()
df.reset_index(inplace = True)

In [7]:
#Combining the all reviews by drugname
df1 = pd.DataFrame(df.groupby('drugName')['review'].apply(' '.join).reset_index())
df1.head()

,drugName,review
0,A + D Cracked Skin Relief,"""I have severe cracked skin on my hands. Ive tried many different products - my skin is extremely sensitive. This product helps heal my skin, does not sting and is not greasy (important when using for hands). This is the best product Ive found for my condition, but it is hard to find - my drugstore cant even order it for me anymore."""
1,A / B Otic,"""It numbs the pain. It makes my ear feel heavier - but at least the throbbing subsides. I always feel drowsy after the dosage. With an ear infection - sleeping is good!"" ""These drops helped me so much. I was in severe pain for 8 days before I finally found a doctor who would prescribe them to me. I went to the ER 5 days in a row before I got any relief."""
2,Abacavir / lamivudine,"""Epzicom, Reyataz AND Norvir triple combination. It is expensive, but ALL hiv medications are expensive and there are programs to help pay for the medicine if you have a low wage since most of these HIV medicines cost between $3000 and $10000 or more. This one in combination with Reyataz and Norvir has brought my CD4 level up to 1100 and an undetectable viral load. I do have side effects from ..."
3,Abacavir / lamivudine / zidovudine,"""I was diagnosed back in 2008 and have been on Trizivir since. My specialist mentioned it is not the newest medicine but it does the trick. One and a half years after starting it I was undetectable and have been since."""
4,Abatacept,"""After rashes, pain, nausea, and elevated liver enzymes on Remicade, then Humira, the doc switched me to Orencia. Felt very tired the evening of the loading dose, but not ill. Now I feel better than I have felt in 4 years! LOVE this stuff!"" ""After the first week of using Orencia, I felt more energized. Before I would wake up feeling tired, and could barely move. I was stiff and could not go ou..."


In [8]:
#Counting the words present in review
df1['review_wordcount'] = df1['review'].str.split().str.len()

In [9]:
#Sorting the drugs in ascending order based on no of words
sorted = df1.sort_values('review_wordcount', ascending=False)
sorted.head()

,drugName,review,review_wordcount
1514,Nexplanon,""" I have been on this for 2 1/2 years now. I am going to have it removed because I have gained weight using it, my periods are heavier and longer with clots, I have become very moody, anxious and angry at the very slightest thing which I wasnt before. I get very terrible migraines with aura. I spoke to my doctor and she said she does not think these side effects could be due to the nexplanon. ...",267069
854,Ethinyl estradiol / norethindrone,""" Im on my 2nd round of necon 1/35. I started taking it because the doctor thinks I have endometriosis, but no one is really sure. My periods are extremely painful and I lose my appetite completely. The side effects of necon are the same as symptoms of pregnancy which is kind of alarming. Im also even more depressed and anxious than I was before I started taking it. I take it without the place...",186537
1248,Levonorgestrel,""" I took levonorgestrel under the brand Postinor 2 after an incident with my boyfriend. My boyfriends semen got over my vaginal area and some of it did get inside me (shallow penetration) and this was during my ovulation period. Took the first pill in 43h, second pill 12h later (postinor requires you to eat 2 pills 12h apart). 1 week after taking the pills I had a lot of vaginal discharge and ...",158176
1940,Sertraline,"""(Sertraline) is the most worst depression treatment for my son. The first time when my son had started to take these pills he was feeling normal but one day he started to have unusual pains which he couldnt describe in the chest which was everyday. Because of the pain he was going to A&amp;E for a check up twice in a week and shockingly nothing had came up. But that pain took my precious sons...",149151
855,Ethinyl estradiol / norgestimate,""" Im 19 years old and started taking this birth control after my first one worked perfectly for a few months, until I began to get my period every 8 days. Ive been on this pill for about two weeks and I already hate it. Im constantly moody, depressed and over emotional. Its effecting my relationship between my family, friends and boyfriend. Doesnt help with acne, Ive never had acne till I bega...",147830


In [10]:
#Count the average rating of each drug
df2 = df.groupby('drugName')['rating'].mean().reset_index()
df2.head()

,drugName,rating
0,A + D Cracked Skin Relief,10.00000
1,A / B Otic,10.00000
2,Abacavir / lamivudine,10.00000
3,Abacavir / lamivudine / zidovudine,9.00000
4,Abatacept,7.26087


In [11]:
#Count the average useful count of each drug
df3 = pd.DataFrame(df.groupby('drugName')['usefulCount'].mean().round().reset_index())
df3.head()

,drugName,usefulCount
0,A + D Cracked Skin Relief,6.0
1,A / B Otic,19.0
2,Abacavir / lamivudine,4.0
3,Abacavir / lamivudine / zidovudine,1.0
4,Abatacept,48.0


In [12]:
#Combining the all conditions in single row based on drug name 
drug_condition = pd.DataFrame(df.groupby('drugName')['condition'].apply(lambda x: list(np.unique(x)))).reset_index()
drug_condition.tail()

,drugName,condition
2330,Zyrtec-D,"[4</span> users found this comment helpful., Allergic Rhinitis]"
2331,Zyvox,"[Bacteremia, Bacterial Infection, Methicillin-Resistant Staphylococcus Aureus Infection, Pneumonia, Skin and Structure Infection]"
2332,ZzzQuil,[Insomnia]
2333,femhrt,"[0</span> users found this comment helpful., Acne, Menstrual Disorders, Postmenopausal Symptoms]"
2334,pHisoHex,"[Skin Disinfection, Preoperative]"


In [13]:
import nltk
nltk.download('punkt')
df4 = pd.DataFrame(df.groupby('drugName')['condition'].apply(' '.join).reset_index())
df4.head()
df4['condition'] = df4.apply(lambda row: nltk.word_tokenize(row['condition']), axis=1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
#Combining the col and make the final dataset in which all reviews are in single row 
data = pd.concat([df1.drugName,drug_condition.condition,df1.review, df2.rating, df3.usefulCount, df1.review_wordcount], axis = 1)
data.tail()

,drugName,condition,review,rating,usefulCount,review_wordcount
2330,Zyrtec-D,"[4</span> users found this comment helpful., Allergic Rhinitis]","""had tried claritin-d, and others prior to finding Zyrtec-D. I have now used it for a number of years, and find it difficult to breath without it. Works very well for me, while nothing else that I could find has."" ""This medicine works better than any other allergy medication with decongestion that I have ever tried.""",9.000000,13.0,56
2331,Zyvox,"[Bacteremia, Bacterial Infection, Methicillin-Resistant Staphylococcus Aureus Infection, Pneumonia, Skin and Structure Infection]","""7 day treatment worked well on an infection that lasted over 5 months"" ""After having arthroscopic knee surgery, I developed a persistent skin infection (cellulitis) at the surgery site. I spent a month on different antibiotics that didnt clear the infection (cephalexin, clindamycin, and ciprofloxacin) and was finally admitted to the hospital for another surgery and treatment with IV antibioti...",8.714286,21.0,1494
2332,ZzzQuil,[Insomnia],"""Gave me rapid heart beats !!!!!!!!!"" ""I purchased the zzzquil and it did not work not 1 bit , I would not recommended medication to anyone..."" ""I usually work a 10 hr shift in the afternoon but my boss has changed my Saturday shift to 5am-1pm. We live on a farm and have 3 outside dogs that like to bark at night. The first morning shift I only got 2 hrs sleep because of our dogs. I purchased Z...",2.000000,7.0,139
2333,femhrt,"[0</span> users found this comment helpful., Acne, Menstrual Disorders, Postmenopausal Symptoms]","""femhrt took away my hot flashes within a week but the acne was unbearable. Stopped taking it."" ""Femhrt worked wonderful. Had to wait a couple of months for the full effect so give it time. The only downfall is that it is expensive and does not have a generic."" ""Hot flashes completely gone in less than 2 weeks. Works great."" ""I was on femHRT for four months and had BV (bacterial vaginitis) all...",6.500000,14.0,263
2334,pHisoHex,"[Skin Disinfection, Preoperative]","""Before Phisohex could only be purchased by prescription, I used it regullarly for feminine hygenine. The product worked perfect, was long lasting, and I never had any side affects. I used Phisohex from 1966 until, much to my dismay, it was taken off my druggist over the counter shelves and could only be purchased with a prescription. Would still be using it if I could. Wonderful solution!""",10.000000,0.0,67


In [16]:
#Import the textblob for calculate the polarity
from textblob import TextBlob
data['polarity'] = data['review'].apply(lambda x: TextBlob(x).sentiment.polarity)
data.tail()

,drugName,condition,review,rating,usefulCount,review_wordcount,polarity
2329,Zyrtec,"[7</span> users found this comment helpful., 9</span> users found this comment helpful., Allergic Rhinitis, Urticaria]","""52 y/o Well I have had hives for 2 almost 3 weeks now on my face scalp neck..because undermined and I never had them before. I was put on pred-Metra-soul pack that I quit due to the extreme side effects. A sedative. I used a heap of ear and eye drops. Was taking Benadryll. I just wanted to claw my skin off..the welts were painful and then became HORRIBLY insanely itchy. One or three would go ...",7.141304,25.0,5795,0.091117
2330,Zyrtec-D,"[4</span> users found this comment helpful., Allergic Rhinitis]","""had tried claritin-d, and others prior to finding Zyrtec-D. I have now used it for a number of years, and find it difficult to breath without it. Works very well for me, while nothing else that I could find has."" ""This medicine works better than any other allergy medication with decongestion that I have ever tried.""",9.000000,13.0,56,0.015000
2331,Zyvox,"[Bacteremia, Bacterial Infection, Methicillin-Resistant Staphylococcus Aureus Infection, Pneumonia, Skin and Structure Infection]","""7 day treatment worked well on an infection that lasted over 5 months"" ""After having arthroscopic knee surgery, I developed a persistent skin infection (cellulitis) at the surgery site. I spent a month on different antibiotics that didnt clear the infection (cephalexin, clindamycin, and ciprofloxacin) and was finally admitted to the hospital for another surgery and treatment with IV antibioti...",8.714286,21.0,1494,-0.048765
2332,ZzzQuil,[Insomnia],"""Gave me rapid heart beats !!!!!!!!!"" ""I purchased the zzzquil and it did not work not 1 bit , I would not recommended medication to anyone..."" ""I usually work a 10 hr shift in the afternoon but my boss has changed my Saturday shift to 5am-1pm. We live on a farm and have 3 outside dogs that like to bark at night. The first morning shift I only got 2 hrs sleep because of our dogs. I purchased Z...",2.000000,7.0,139,0.078283
2333,femhrt,"[0</span> users found this comment helpful., Acne, Menstrual Disorders, Postmenopausal Symptoms]","""femhrt took away my hot flashes within a week but the acne was unbearable. Stopped taking it."" ""Femhrt worked wonderful. Had to wait a couple of months for the full effect so give it time. The only downfall is that it is expensive and does not have a generic."" ""Hot flashes completely gone in less than 2 weeks. Works great."" ""I was on femHRT for four months and had BV (bacterial vaginitis) all...",6.500000,14.0,263,0.067333


In [17]:
#Import the csv file which has 1.77 lakhs side effects and symptoms
side_effects = pd.read_csv('drive/My Drive/CSV_Files/sideeffects.csv', encoding = 'ISO-8859-1')
side_effects = side_effects['Side Effect Name']

In [18]:
#Function to find side effect from reviews

def Side_Effect(Review):
    s_word=[]
    for w in side_effects:
        if w in Review:
            s_word.append(w)
    return s_word
    

Sf = lambda x: Side_Effect(x)

In [19]:
#Store the side effects into dataframe
#As there are almost 2400 records so it will take approx 1.5 hrs to execute this function
data['Side_Effect'] = data.review.apply(Sf)

In [20]:
#Removing the duplicate side effects
data['Side_Effect'] = data['Side_Effect'].map(lambda x: list(set(x)))

In [21]:
#Calculating the effectiveness using drug rating and polarity of reviews
data['effectiveness'] = data['rating']*0.07 + data['polarity']*0.3
data['effectiveness'] = data['effectiveness'].map(lambda n: '{:.2%}'.format(n))

In [22]:
#Functions to find positive words in the review

import string
def Effective_Review(Review):
    Pos_Word=[]
    Review=Review.split(".")
    for word in Review:
        tb = TextBlob(word)
        if tb.sentiment.polarity >= 0.7:
            Pos_Word.append(word)
    return Pos_Word

Effective = lambda x: Effective_Review(x)

In [23]:
data['Pos_Sent'] = data.review.apply(Effective)


In [ ]:
#Functions to find negative words in the review
def Not_Effective_Review(Review):
    Neg_Word=[]
    Review=Review.split(".")
    for word in Review:
        tb = TextBlob(word)
        if tb.sentiment.polarity <= -0.7:
            Neg_Word.append(word)
    return Neg_Word

Not_Effective = lambda x: Not_Effective_Review(x)

In [ ]:
data['Neg_Sent'] = data.review.apply(Not_Effective)

In [26]:
data.tail()

,drugName,condition,review,rating,usefulCount,review_wordcount,polarity,Side_Effect,effectiveness,Pos_Sent,Neg_Sent
2329,Zyrtec,"[7</span> users found this comment helpful., 9</span> users found this comment helpful., Allergic Rhinitis, Urticaria]","""52 y/o Well I have had hives for 2 almost 3 weeks now on my face scalp neck..because undermined and I never had them before. I was put on pred-Metra-soul pack that I quit due to the extreme side effects. A sedative. I used a heap of ear and eye drops. Was taking Benadryll. I just wanted to claw my skin off..the welts were painful and then became HORRIBLY insanely itchy. One or three would go ...",7.141304,25.0,5795,0.091117,"[red blotches, cough, Urticaria, stress, ache, aversion, anger, fall, lethargy, nightmare, withdraw, fever, bite, sneeze, pain, fatigue, groggy, flare, drowsiness, weight gain, swelling, elation, sleepy, headache, welt, scratch, sniffle, tired, autoimmune disorder, idiopathic urticaria]",52.72%,"[ Felt good! ITCHING STOPPED, No drowsiness is the best part, Anyhow now im researching what to take for mango skin allergy, I have Allegra which isnt as good and since I dont work now Im going to take Zyrtec for it after reading the reviews on it, I give Zyrtec 8 stars BC it is a great otc fix for allergies, They were very effective on me, feeling great, Great product, I am happy to say...","[the welts were painful and then became HORRIBLY insanely itchy, Terrible, I just got done with a horrible case from penicillin, HATE HATE HATE THE PACKAGING, The horrible migraines were withdrawal symptoms! I immediately stopped giving it to my kids and taking it myself, It took a couple weeks to get through the worst of it, but I am still itching after a month]"
2330,Zyrtec-D,"[4</span> users found this comment helpful., Allergic Rhinitis]","""had tried claritin-d, and others prior to finding Zyrtec-D. I have now used it for a number of years, and find it difficult to breath without it. Works very well for me, while nothing else that I could find has."" ""This medicine works better than any other allergy medication with decongestion that I have ever tried.""",9.000000,13.0,56,0.015000,[],63.45%,[],[]
2331,Zyvox,"[Bacteremia, Bacterial Infection, Methicillin-Resistant Staphylococcus Aureus Infection, Pneumonia, Skin and Structure Infection]","""7 day treatment worked well on an infection that lasted over 5 months"" ""After having arthroscopic knee surgery, I developed a persistent skin infection (cellulitis) at the surgery site. I spent a month on different antibiotics that didnt clear the infection (cephalexin, clindamycin, and ciprofloxacin) and was finally admitted to the hospital for another surgery and treatment with IV antibioti...",8.714286,21.0,1494,-0.048765,"[Sepsis, chill, fever, headache, pain, surgery, swelling, tired, Influenza, vomiting, nausea, dizziness, diarrhea, ache, bone infection, sleepy, wound, open wound]",59.54%,"[ If this works then its an amazing, wonderful drug, It was however very effective]","[ It is a very unpleasant feeling, It was the worst 2 weeks of my life, Weakness, I felt horrible, My doctor keeps on eye on me but the brown tongue and funky taste is disgusting]"
2332,ZzzQuil,[Insomnia],"""Gave me rapid heart beats !!!!!!!!!"" ""I purchased the zzzquil and it did not work not 1 bit , I would not recommended medication to anyone..."" ""I usually work a 10 hr shift in the afternoon but my boss has changed my Saturday shift to 5am-1pm. We live on a farm and have 3 outside dogs that like to bark at night. The first morning shift I only got 2 hrs sleep because of our dogs. I purchased Z...",2.000000,7.0,139,0.078283,[fall],16.35%,[],[]
2333,femhrt,"[0</span> users found this comment helpful., Acne, Menstrual Disorders, Postmenopausal Symptoms]","""femhrt took away my hot flashes within a week but the acne was unbearable. Stopped taking it."" ""Femhrt worked wonderful. Had to wait a couple of months for the full effect so give it time. The only dow

In [ ]:
#Removing unnecessary brackets from each col
data['condition'] = data['condition'].apply(lambda x:', '.join([i for i in x 
                                                  if i not in string.punctuation]))

In [ ]:
data['Side_Effect'] = data['Side_Effect'].apply(lambda x:', '.join([i for i in x 
                                                  if i not in string.punctuation]))

In [ ]:
data['Pos_Sent'] = data['Pos_Sent'].apply(lambda x:', '.join([i for i in x 
                                                  if i not in string.punctuation]))

In [ ]:
data['Neg_Sent'] = data['Neg_Sent'].apply(lambda x:', '.join([i for i in x 
                                                  if i not in string.punctuation]))

In [31]:
data.tail()

,drugName,condition,review,rating,usefulCount,review_wordcount,polarity,Side_Effect,effectiveness,Pos_Sent,Neg_Sent
2329,Zyrtec,"7</span> users found this comment helpful., 9</span> users found this comment helpful., Allergic Rhinitis, Urticaria","""52 y/o Well I have had hives for 2 almost 3 weeks now on my face scalp neck..because undermined and I never had them before. I was put on pred-Metra-soul pack that I quit due to the extreme side effects. A sedative. I used a heap of ear and eye drops. Was taking Benadryll. I just wanted to claw my skin off..the welts were painful and then became HORRIBLY insanely itchy. One or three would go ...",7.141304,25.0,5795,0.091117,"red blotches, cough, Urticaria, stress, ache, aversion, anger, fall, lethargy, nightmare, withdraw, fever, bite, sneeze, pain, fatigue, groggy, flare, drowsiness, weight gain, swelling, elation, sleepy, headache, welt, scratch, sniffle, tired, autoimmune disorder, idiopathic urticaria",52.72%,"Felt good! ITCHING STOPPED, No drowsiness is the best part, Anyhow now im researching what to take for mango skin allergy, I have Allegra which isnt as good and since I dont work now Im going to take Zyrtec for it after reading the reviews on it, I give Zyrtec 8 stars BC it is a great otc fix for allergies, They were very effective on me, feeling great, Great product, I am happy to say,...","the welts were painful and then became HORRIBLY insanely itchy, Terrible, I just got done with a horrible case from penicillin, HATE HATE HATE THE PACKAGING, The horrible migraines were withdrawal symptoms! I immediately stopped giving it to my kids and taking it myself, It took a couple weeks to get through the worst of it, but I am still itching after a month"
2330,Zyrtec-D,"4</span> users found this comment helpful., Allergic Rhinitis","""had tried claritin-d, and others prior to finding Zyrtec-D. I have now used it for a number of years, and find it difficult to breath without it. Works very well for me, while nothing else that I could find has."" ""This medicine works better than any other allergy medication with decongestion that I have ever tried.""",9.000000,13.0,56,0.015000,,63.45%,,
2331,Zyvox,"Bacteremia, Bacterial Infection, Methicillin-Resistant Staphylococcus Aureus Infection, Pneumonia, Skin and Structure Infection","""7 day treatment worked well on an infection that lasted over 5 months"" ""After having arthroscopic knee surgery, I developed a persistent skin infection (cellulitis) at the surgery site. I spent a month on different antibiotics that didnt clear the infection (cephalexin, clindamycin, and ciprofloxacin) and was finally admitted to the hospital for another surgery and treatment with IV antibioti...",8.714286,21.0,1494,-0.048765,"Sepsis, chill, fever, headache, pain, surgery, swelling, tired, Influenza, vomiting, nausea, dizziness, diarrhea, ache, bone infection, sleepy, wound, open wound",59.54%,"If this works then its an amazing, wonderful drug, It was however very effective","It is a very unpleasant feeling, It was the worst 2 weeks of my life, Weakness, I felt horrible, My doctor keeps on eye on me but the brown tongue and funky taste is disgusting"
2332,ZzzQuil,Insomnia,"""Gave me rapid heart beats !!!!!!!!!"" ""I purchased the zzzquil and it did not work not 1 bit , I would not recommended medication to anyone..."" ""I usually work a 10 hr shift in the afternoon but my boss has changed my Saturday shift to 5am-1pm. We live on a farm and have 3 outside dogs that like to bark at night. The first morning shift I only got 2 hrs sleep because of our dogs. I purchased Z...",2.000000,7.0,139,0.078283,fall,16.35%,,
2333,femhrt,"0</span> users found this comment helpful., Acne, Menstrual Disorders, Postmenopausal Symptoms","""femhrt took away my hot flashes within a week but the acne was unbearable. Stopped taking it."" ""Femhrt worked wonderful. Had to wait a couple of months for the full effect so give it time. The only downfall is that it is expensive and d

In [32]:
data=data.drop(["review_wordcount","usefulCount"],axis=1)

In [33]:
data.tail()

,drugName,condition,review,rating,polarity,Side_Effect,effectiveness,Pos_Sent,Neg_Sent
2329,Zyrtec,"7</span> users found this comment helpful., 9</span> users found this comment helpful., Allergic Rhinitis, Urticaria","""52 y/o Well I have had hives for 2 almost 3 weeks now on my face scalp neck..because undermined and I never had them before. I was put on pred-Metra-soul pack that I quit due to the extreme side effects. A sedative. I used a heap of ear and eye drops. Was taking Benadryll. I just wanted to claw my skin off..the welts were painful and then became HORRIBLY insanely itchy. One or three would go ...",7.141304,0.091117,"red blotches, cough, Urticaria, stress, ache, aversion, anger, fall, lethargy, nightmare, withdraw, fever, bite, sneeze, pain, fatigue, groggy, flare, drowsiness, weight gain, swelling, elation, sleepy, headache, welt, scratch, sniffle, tired, autoimmune disorder, idiopathic urticaria",52.72%,"Felt good! ITCHING STOPPED, No drowsiness is the best part, Anyhow now im researching what to take for mango skin allergy, I have Allegra which isnt as good and since I dont work now Im going to take Zyrtec for it after reading the reviews on it, I give Zyrtec 8 stars BC it is a great otc fix for allergies, They were very effective on me, feeling great, Great product, I am happy to say,...","the welts were painful and then became HORRIBLY insanely itchy, Terrible, I just got done with a horrible case from penicillin, HATE HATE HATE THE PACKAGING, The horrible migraines were withdrawal symptoms! I immediately stopped giving it to my kids and taking it myself, It took a couple weeks to get through the worst of it, but I am still itching after a month"
2330,Zyrtec-D,"4</span> users found this comment helpful., Allergic Rhinitis","""had tried claritin-d, and others prior to finding Zyrtec-D. I have now used it for a number of years, and find it difficult to breath without it. Works very well for me, while nothing else that I could find has."" ""This medicine works better than any other allergy medication with decongestion that I have ever tried.""",9.000000,0.015000,,63.45%,,
2331,Zyvox,"Bacteremia, Bacterial Infection, Methicillin-Resistant Staphylococcus Aureus Infection, Pneumonia, Skin and Structure Infection","""7 day treatment worked well on an infection that lasted over 5 months"" ""After having arthroscopic knee surgery, I developed a persistent skin infection (cellulitis) at the surgery site. I spent a month on different antibiotics that didnt clear the infection (cephalexin, clindamycin, and ciprofloxacin) and was finally admitted to the hospital for another surgery and treatment with IV antibioti...",8.714286,-0.048765,"Sepsis, chill, fever, headache, pain, surgery, swelling, tired, Influenza, vomiting, nausea, dizziness, diarrhea, ache, bone infection, sleepy, wound, open wound",59.54%,"If this works then its an amazing, wonderful drug, It was however very effective","It is a very unpleasant feeling, It was the worst 2 weeks of my life, Weakness, I felt horrible, My doctor keeps on eye on me but the brown tongue and funky taste is disgusting"
2332,ZzzQuil,Insomnia,"""Gave me rapid heart beats !!!!!!!!!"" ""I purchased the zzzquil and it did not work not 1 bit , I would not recommended medication to anyone..."" ""I usually work a 10 hr shift in the afternoon but my boss has changed my Saturday shift to 5am-1pm. We live on a farm and have 3 outside dogs that like to bark at night. The first morning shift I only got 2 hrs sleep because of our dogs. I purchased Z...",2.000000,0.078283,fall,16.35%,,
2333,femhrt,"0</span> users found this comment helpful., Acne, Menstrual Disorders, Postmenopausal Symptoms","""femhrt took away my hot flashes within a week but the acne was unbearable. Stopped taking it."" ""Femhrt worked wonderful. Had to wait a couple of months for the full effect so give it time. The only downfall is that it is expensive and does not have a generic."" ""Hot flashes completely gone in less t

In [34]:
output = pd.concat([data.drugName,data.condition,data.Side_Effect,data.effectiveness],axis = 1)


In [35]:
output.tail()

,drugName,condition,Side_Effect,effectiveness
2329,Zyrtec,"7</span> users found this comment helpful., 9</span> users found this comment helpful., Allergic Rhinitis, Urticaria","red blotches, cough, Urticaria, stress, ache, aversion, anger, fall, lethargy, nightmare, withdraw, fever, bite, sneeze, pain, fatigue, groggy, flare, drowsiness, weight gain, swelling, elation, sleepy, headache, welt, scratch, sniffle, tired, autoimmune disorder, idiopathic urticaria",52.72%
2330,Zyrtec-D,"4</span> users found this comment helpful., Allergic Rhinitis",,63.45%
2331,Zyvox,"Bacteremia, Bacterial Infection, Methicillin-Resistant Staphylococcus Aureus Infection, Pneumonia, Skin and Structure Infection","Sepsis, chill, fever, headache, pain, surgery, swelling, tired, Influenza, vomiting, nausea, dizziness, diarrhea, ache, bone infection, sleepy, wound, open wound",59.54%
2332,ZzzQuil,Insomnia,fall,16.35%
2333,femhrt,"0</span> users found this comment helpful., Acne, Menstrual Disorders, Postmenopausal Symptoms","birth control, bedridden, fall, pain",47.52%


In [36]:
output.to_csv("Output.csv")

In [37]:
data.to_csv("All_Output.csv")